In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pd_support import *
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import interp1d
from datetime import datetime
from numba import njit
import itertools
import warnings
from calculations_support import *

In [2]:
mc, mc_dt = read_df_csv("../data_csv/random_movement_sync_mc.csv")
cart_df = pd.read_csv("../data_csv/random_movement_sync.csv")

In [3]:
mc = add_datetime_col(mc, mc_dt, "seconds")
cart_df = add_datetime_diff(cart_df, mc_dt,"sync", "mils", True)
mc["time"][0]


5412


Timestamp('2022-06-23 09:53:21.904000')

In [4]:
"""getting initial values of motion capture data"""

# marker_cen = np.array(mc[["center_x", "center_y", "center_z"]].iloc[0]).T
marker_xvec = np.array(mc[["xdir_x", "xdir_y", "xdir_z"]].iloc[0]).T
marker_zvec = np.array(mc[["zdir_x", "zdir_y", "zdir_z"]].iloc[0]).T
marker_org = np.array(mc[["org_x", "org_y", "org_z"]].iloc[0]).T
marker_org[0]

0.055481

In [5]:
def calculate_rotmat(xdir,zdir,org):
    """
    this function calculates rotation matrix
    """
    v1 = xdir - org #v1
    v2 = zdir - org #v2

    vxnorm = v1/np.linalg.norm(v1)

    vzcap = v2 - (vxnorm.T @ v2) * vxnorm
    vznorm = vzcap/ np.linalg.norm(vzcap)

    vynorm = np.cross(vznorm.T[0], vxnorm.T[0]).reshape(3,1)
    rotMat = np.hstack((vxnorm, vynorm, vznorm))
    return rotMat

In [6]:
v1 = marker_xvec - marker_org #v1
v2 = marker_zvec - marker_org #v2

v1 = v1.reshape(3,1)
v2 = v2.reshape(3,1)
v3 = marker_org.reshape(3,1)

In [7]:
rot_mat = calculate_rotmat(v1, v2, v3)

In [8]:
"""circumfrence of the wheel"""

cir_wheel = np.degrees(2*np.pi*(47.5/1000))
cir_wheel
radius = 47.5/1000

lx = 79 #half of the distance between the wheels
ly = 122.5/2

lx = lx/1000
ly = ly/1000

In [9]:
"""calculating angles"""
""" angular velocity"""
cart_df["av_fr"] = (cart_df["e_fr"]*0.09).diff()/0.01
cart_df["av_fl"] = (cart_df["e_fl"]*0.09).diff()/0.01
cart_df["av_rr"] = (cart_df["e_rr"]*0.09).diff()/0.01
cart_df["av_rl"] = (cart_df["e_rl"]*0.09).diff()/0.01
cart_df['av_fr'] = cart_df["av_fr"].fillna(0)
cart_df['av_fl'] = cart_df["av_fl"].fillna(0)
cart_df['av_rr'] = cart_df["av_rr"].fillna(0)
cart_df['av_rl'] = cart_df["av_rl"].fillna(0)

#converting them to radians
cart_df['av_fr'] = cart_df["av_fr"]* np.pi/180
cart_df['av_fl'] = cart_df["av_fl"]* np.pi/180
cart_df['av_rr'] = cart_df["av_rr"]* np.pi/180
cart_df['av_rl'] = cart_df["av_rl"]* np.pi/180

# cart_df["av_fl"]

In [14]:

_b1 = range(88, 91, 1)
_b2 = range(88, 91, 1)
_b3 = range(88, 91, 1)
_b4 = range(88, 91, 1)
_a1 = range(50, 53, 1)
_a2= range(50, 53, 1)
_a3= range(50, 53, 1)
_a4 = range(50, 53, 1)

pseudo_mat, iter_array, length_of_iterables = combinations_of_pseudo_mat(_b1, _b2, _b3, _b4, _a1, _a2, _a3, _a4, radius)
pseudo_mat.shape
length_of_iterables

6561

In [11]:
cart_inf = interpolate_data(cart_df, mc, ["av_fr", "av_fl", "av_rr", "av_rl"])

In [12]:
cart_inf

,av_fr,av_fl,av_rr,av_rl,time
0,0.0,0.0,0.0,0.0,2022-06-23 09:53:21.904
1,0.0,0.0,0.0,0.0,2022-06-23 09:53:21.914
2,0.0,0.0,0.0,0.0,2022-06-23 09:53:21.924
3,0.0,0.0,0.0,0.0,2022-06-23 09:53:21.934
4,0.0,0.0,0.0,0.0,2022-06-23 09:53:21.944
...,...,...,...,...,...
5401,0.0,0.0,0.0,0.0,2022-06-23 09:54:15.914
5402,0.0,0.0,0.0,0.0,2022-06-23 09:54:15.924
5403,0.0,0.0,0.0,0.0,2022-06-23 09:54:15.934
5404,0.0,0.0,0.0,0.0,2022-06-23 09:54:15.944


In [13]:
iterate_get_diff(pseudo_mat, cart_inf, ["av_fr", "av_fl", "av_rr", "av_rl"], mc, ["org_z", "org_x"])

0      -0.041580
1      -0.041599
2      -0.041599
3      -0.041581
4      -0.041580
          ...   
5401    0.057140
5402    0.057139
5403    0.057147
5404    0.057147
5405    0.057155
Length: 5406, dtype: float64
